# Setup

In [1]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import seaborn as sns
import json
import numpy as np
from datetime import datetime
from backtesting_utils import (make_results,
                               drawdown, merge,
                               reduce_daily_trades,
                               disable_code_cells,
                               add_open_trades,
                               get_streak_runs,
                               map_risk_to_streak,
                               result_by_close_given_risk,
                               challenge_pass,
                               plot_pos)

import cufflinks as cf
cf.set_config_file(offline=True, offline_show_link=False, theme='white')
cf_colors = ["#ff3838", "#38ff38", "#3838ff", "#ffdf28", "#28ffdf",
             "#df28ff", "#08FF7C", "#ad9042", "#42ad90"]

print(f"Last run: {datetime.now().isoformat()}")

Last run: 2022-04-13T18:52:52.638135


In [2]:
with open('../input/prepro-daniel-2/test_prepro_daniel.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

In [3]:
disable_code_cells()

***
# Analisi Risultati

Partiamo ipotizzando di seguire "alla lettera" (ammesso un certo margine di errore) i segnali e iniziamo con lo studiare le due varianti più importanti:
- Alla lettera, chiudendo l'intera posizione a TP1
- Alla lettera, chiudendo il 60% della posizione a TP1 e il 40% a TP2

In [4]:
def base_filter(df: pd.DataFrame) -> pd.DataFrame:
    df = df.drop(df[df.sl_delta / df.tick_size / 10 > 50].index, axis=0)
    df = df.drop(df[df.sl_delta / df.tick_size / 10 < 5].index, axis=0)
    df = df[df.open != "2021-12-22 05:53"] # outlier
    df = df[df.open != "2021-10-5 13:22"] # outlier
    df = df[df.open != "2021-02-05 08:43"] # outlier
    df = df[df.open != "2022-2-14 09:13"] # outlier
    return df

In [5]:
df64 = make_results(data, partials=[0.6, 0.4], only=['symbol', 'open', 'close', 'result', 'side'], cleanse=base_filter)
df1 = make_results(data, partials=[1], only=['symbol','open', 'close', 'result', 'side'], cleanse=base_filter)

In [6]:
df64_by_c = df64.set_index('close').sort_index()
df1_by_c = df1.set_index('close').sort_index()
df64_by_o = df64.set_index('open').sort_index()
df1_by_o = df1.set_index('open').sort_index()

In [7]:
merged = merge({'60/40': df64_by_c[['result']], '100': df1_by_c[['result']]})   
merged.cumsum().iplot(kind='spread', connectgaps=True, interpolation='hv', 
                      title='Risultato cumulativo (R) [by Close]', width=1.5, legend='top',
                      yTitle='R', fill=False, zerolinecolor='grey', colors=cf_colors)

In [8]:
df = merged.copy().groupby(pd.Grouper(freq='M')).sum().reset_index()
df['close'] = df.close.dt.strftime('%b %Y')
df.iplot(kind='bar', x='close', opacity=1, title='Risultato mensile (R) [by Close]',
         yTitle='R', zerolinecolor='grey', colors=cf_colors, xTitle='month', legend='top')

In [9]:
df = merged.copy().groupby(pd.Grouper(freq='D')).sum()
df = df[df!=0].dropna(how='all').reset_index()
n_days = (df.iloc[-1, 0] - df.iloc[0, 0]).days
df['close'] = df.close.dt.strftime('%d %b %Y')
df.iplot(kind='bar', x='close', opacity=1, yTitle='R', zerolinecolor='grey',
         title=f'Risultato giornaliero (R) [by Close] [non_zero={len(df)}/{n_days}] [worked={len(df)/(n_days*.7):.0%}]',
         colors=cf_colors, xTitle='date', legend='top')

In [10]:
merged = merge({'60/40': drawdown(df64_by_c),
               '100': drawdown(df1_by_c)})

merged.reset_index(drop=True).iplot(kind='scatter', connectgaps=True, interpolation='hv', colors=cf_colors, fill=False,
                                    title='Drawdown (R) [by Close]', xTitle='index', yTitle='R', legend='bottom', width=1.5)

## Analisi direzionale

In [11]:
df = df1.copy()
threshold = 0.1
longs = df[df.side == 'BUY']
shorts = df[df.side == 'SELL']
longs_won = longs[df.result > threshold]
shorts_won = shorts[df.result > threshold]

notes = {0: (f'won={len(longs_won):.0f}/{len(longs):.0f} ({len(longs_won)/len(longs):.1%}) | '
             f'avg_won_R={longs_won.result.mean():.2f}'),

         1: (f'won={len(shorts_won):.0f}/{len(shorts):.0f} ({len(shorts_won)/len(shorts):.1%}) | '
            f'avg_won_R={shorts_won.result.mean():.2f}')}

df.groupby('side').sum().iplot(kind='bar', colors=cf_colors, opacity=1, yTitle='R',
                               zerolinecolor='grey', textangle=0, fontcolor='black', fontsize=14, annotations=notes, 
                               title=f'Risultato cumulativo in base alla direzione (R) [threshold={threshold}]')

In [12]:
df = df64.copy()
threshold = 0.1
longs = df[df.side == 'BUY']
shorts = df[df.side == 'SELL']

df['longs'] = longs.result
df['shorts'] = shorts.result

df.set_index('close').groupby(pd.Grouper(freq='M')).sum()[['longs', 'shorts']].iplot(
    kind='bar', barmode='group', colors=cf_colors, opacity=1, yTitle='R', legend='top', zerolinecolor='grey',
    title=f'Risultati cumulativi in base alla direzione, per mese (R) [threshold={threshold}]')

In [13]:
df = df64.copy()
threshold = 0.1
longs = df[df.side == 'BUY']
shorts = df[df.side == 'SELL']

df['longs_won'] = longs.result.apply(lambda x: 1 if x > threshold else 0)
df['shorts_won'] = shorts.result.apply(lambda x: 1 if x > threshold else 0)
df['longs'] = (df.side == 'BUY').astype(int)
df['shorts'] = (df.side == 'SELL').astype(int)

by_M = df.set_index('close').groupby(pd.Grouper(freq='M')).sum()
by_M['longs_won_%'] = 100 * by_M.longs_won / by_M.longs
by_M['shorts_won_%'] = 100 * by_M.shorts_won / by_M.shorts

by_M[['longs', 'longs_won', 'shorts', 'shorts_won']].iplot(kind='bar', barmode='group', colors=cf_colors, 
                                                           opacity=1, yTitle='trades', legend='top',
                                                           title=f'Esito in base alla direzione, per mese [threshold={threshold}]')
by_M[['longs_won_%', 'shorts_won_%']].iplot(kind='bar', barmode='group', colors=cf_colors, 
                                            opacity=1, yTitle='%', legend='top', zerolinecolor='grey',
                                            title=f'Winrate in base alla direzione, per mese [threshold={threshold}]')

In [14]:
df = df64.copy()
threshold = 0.1
longs = df[df.side == 'BUY']
shorts = df[df.side == 'SELL']

df['longs_won'] = longs.result.apply(lambda x: 1 if x > threshold else 0)
df['shorts_won'] = shorts.result.apply(lambda x: 1 if x > threshold else 0)
df['longs'] = (df.side == 'BUY').astype(int)
df['shorts'] = (df.side == 'SELL').astype(int)

df = df.set_index('open').sort_index()

df['counter'] = 1
df['counter'] = df[['counter']].groupby(pd.Grouper(freq='D')).transform(lambda x: x.cumsum())

df = df.reset_index().set_index('close').groupby('counter').sum()
df['longs_won_%'] = 100 * df.longs_won / df.longs
df['shorts_won_%'] = 100 * df.shorts_won / df.shorts

df

,result,longs_won,shorts_won,longs,shorts,longs_won_%,shorts_won_%
counter,,,,,,,
1,18.159809,112.0,32.0,203,78,55.172414,41.025641
2,10.299994,54.0,14.0,107,34,50.467290,41.176471
3,2.018651,24.0,5.0,52,8,46.153846,62.500000
4,-1.809257,9.0,2.0,23,4,39.130435,50.000000
5,-3.080640,7.0,0.0,11,2,63.636364,0.000000
6,-0.997283,0.0,0.0,1,0,0.000000,NaN


In [15]:
gj_daily = pd.read_csv('../input/gbpjpy-rates/daily_GBPJPY.csv', index_col=0)
gj_weekly =  pd.read_csv('../input/gbpjpy-rates/weekly_GBPJPY.csv', index_col=0)
gj_monthly =  pd.read_csv('../input/gbpjpy-rates/monthly_GBPJPY.csv', index_col=0)

final = []
for gj, by in zip((gj_daily, gj_weekly, gj_monthly), ('1D', '1W', '1M')):

    gj['mkt_side'] = (gj.close > gj.open).map({True: 'LONG', False: 'SHORT'})
    gj['time'] = pd.to_datetime(gj.time).dt.normalize()
    df = df64.copy()[df1.symbol=='GBPJPY'].set_index('close')

    df = df.groupby([pd.Grouper(freq=by), df.side]).sum()
    df = pd.merge_asof(df, gj.set_index('time')[['mkt_side']], left_on='close', right_index=True, tolerance=pd.Timedelta(by))
    df = df.groupby(['side', 'mkt_side']).sum().reset_index()
    
    df['toplot'] = df.side + ' in a ' + df.mkt_side
    final.append(df[['result', 'toplot']].set_index('toplot').T.set_index(pd.Series(by)))
    
pd.concat(final).iplot(kind='bar', colors=cf_colors, yTitle='R', opacity=1, zerolinecolor='grey', legend='top',
                       title='Risultati in base alla direzione, in base alla direzione della candela di riferimento')

### Proviamo ad analizzare cosa succede sfruttando questa asimmmetria

In [16]:
df = df64.copy()
df['result_no_sell'] = df.result * df.side.map({'BUY': 1, 'SELL': 0})
df['result_inv_sell'] = (df.result * df.side.map({'BUY': 1, 'SELL': -1})).apply(lambda x: -1 if x < -0.9 else x)
df.set_index('close').sort_index().cumsum().iplot(y=['result', 'result_no_sell', 'result_inv_sell'],
                                                  title='Risultato cumulativo (R) [by Close]', width=1.5,
                                                  legend='top', yTitle='R', fill=False, zerolinecolor='grey', 
                                                  colors=cf_colors, interpolation='hv')

In [17]:
df = df.set_index('close').copy().groupby(pd.Grouper(freq='M')).sum().reset_index()
df['close'] = df.close.dt.strftime('%b %Y')
df.iplot(kind='bar', x='close', y=['result', 'result_no_sell', 'result_inv_sell'], opacity=1,
         title='Risultato mensile (R) [by Close]', yTitle='R', zerolinecolor='grey',
         colors=cf_colors, xTitle='month', legend='top')

## Analisi relativa alla frequenza dei trades

In [18]:
df = df1_by_o.copy().groupby(pd.Grouper(freq='D')).count()
df = df[df!=0].dropna(how='all')
df['result'].iplot(kind='hist', histnorm='probability', colors=cf_colors, opacity=1, xTitle='trades',
                   title='Distribuzione del numero di trades in 1 giorno (frequenza) [by Open]',
                   yTitle='probability', bargroupgap=0.2, linecolor='grey', zerolinecolor='grey')

In [19]:
df['result'] = df1_by_o.copy().groupby(pd.Grouper(freq='D')).sum()['result']
df.groupby('close').sum().loc[1:, 'result'].iplot(kind='bar', opacity=1, colors=cf_colors,
                                                  title='Risultato cumulativo in base al numero di trade giornalieri (R) [by Open]',
                                                  yTitle='R', zerolinecolor='grey')

In [20]:
df = df1_by_o.copy()
df['counter'] = 1
df['counter'] = df[['counter']].groupby(pd.Grouper(freq='D')).transform(lambda x: x.cumsum())
df.groupby('counter').sum().loc[1:, 'result'].iplot(kind='bar', opacity=1, colors=cf_colors,
                                                    title='Risultato cumulativo in base alla posizione del trade nella giornata (R) [by Open]',
                                                    yTitle='R', zerolinecolor='grey')

***
## Limite al numero di trades giornalieri
E' chiaro che il numero di trades giornalieri influisce sul risultato della strategia:
 - Proviamo a limitire il numero massimo di operazioni giornaliere
 - Paragonerò la strategia 60/40 visto che si è rivelata più profittevole

In [21]:
reducer = lambda x, n: reduce_daily_trades(x, n)
variations = {f'60/40_{n}': reducer(df64_by_o, n).reset_index().set_index('close').sort_index() for n in range(1,7)}
merged = merge(variations)

In [22]:
merged.copy().cumsum().iplot(kind='scatter', connectgaps=True, interpolation='hv', 
                      title='Risultato cumulativo (R) [by Close]', width=1.5, legend='top',
                      yTitle='R', fill=False, zerolinecolor='grey', opacity=1, colors=cf_colors)

In [23]:
merged = merge({
    '60/40_2': variations['60/40_2'],
    '60/40_3': variations['60/40_3'],
    '60/40_6': variations['60/40_6']})

In [24]:
df = merged.copy().groupby(pd.Grouper(freq='D')).sum()
df = df[df!=0].dropna(how='all').reset_index()
n_days = (df.iloc[-1, 0] - df.iloc[0, 0]).days
df['close'] = df.close.dt.strftime('%d %b %Y')
df.iplot(kind='bar', x='close', opacity=1, yTitle='R', zerolinecolor='grey', subplots=True,
         title=f'Risultato giornaliero (R) [by Close] [non_zero={len(df)}/{n_days}] [worked={len(df)/(n_days*.7):.0%}]',
         colors=cf_colors, xTitle='date', legend='top', shared_yaxis=True)

In [25]:
merged = merge({
    '60/40_2': drawdown(variations['60/40_2']),
    '60/40_3': drawdown(variations['60/40_3']),
    '60/40_6': drawdown(df64_by_c)})

merged.reset_index(drop=True).iplot(kind='scatter', connectgaps=True, interpolation='hv', colors=cf_colors, fill=False,
                                    title='Drawdown (R) [by Close]', xTitle='index', yTitle='R', legend='bottom', width=1.5)

***
## Proviamo ad unire la limitazione del numero dei trade e la modifica dei trade short

In [26]:
variations = {}
for suff in ('', '_no_sell', '_inv_sell'):
    for n in [2, 3, 6]:
        df = reduce_daily_trades(df64_by_o, n)
        if suff == '':
            pass
        elif suff == '_no_sell':
            df['result'] = df.result * df.side.map({'BUY': 1, 'SELL': 0})
        elif suff == '_inv_sell':
            df['result'] = (df.result * df.side.map({'BUY': 1, 'SELL': -1})).apply(lambda x: -1 if x < -0.9 else x)
        
        variations[f'60/40_{n}{suff}'] = df.reset_index().set_index('close').sort_index()
merged = merge(variations)

In [27]:
merged.copy().cumsum().iplot(kind='scatter', connectgaps=True, interpolation='hv', 
                      title='Risultato cumulativo (R) [by Close]', width=1.5, legend='right',
                      yTitle='R', fill=False, zerolinecolor='grey', opacity=1, colors=cf_colors)

In [28]:
df = merged.copy().groupby(pd.Grouper(freq='D')).sum()
df = df[df!=0].dropna(how='all').reset_index()
n_days = (df.iloc[-1, 0] - df.iloc[0, 0]).days
df['close'] = df.close.dt.strftime('%d %b %Y')
df.iplot(kind='bar', x='close', opacity=1, zerolinecolor='grey', subplots=True, yTitle='',
         title=f'Risultato giornaliero (R) [by Close] [non_zero={len(df)}/{n_days}] [worked={len(df)/(n_days*.7):.0%}]',
         colors=cf_colors, xTitle='date', legend='right', shared_yaxis=True, showlegend=True)

In [29]:
merged_ = merge({f'{k}_dd': drawdown(v) for k, v in variations.items()})

merged_.reset_index(drop=True).iplot(kind='scatter', connectgaps=True, interpolation='hv', colors=cf_colors, fill=False,
                                    title='Drawdown (R) [by Close]', xTitle='index', yTitle='R', legend='right', width=1.5)

***
## Risk Management Dinamico

In [30]:
merged = merge({'60/40_2': add_open_trades(variations['60/40_2'])[['open_trades']],
                '60/40_3': add_open_trades(variations['60/40_3'])[['open_trades']],
                '60/40_6': add_open_trades(variations['60/40_6'])[['open_trades']]}).reset_index(drop=True)

non_one = sum([len(merged[(merged[f'{x}'].notna()) & (merged[f'{x}'] != 1)]) / 
               len(merged[merged[f'{x}'].notna()]) for x in merged.columns]) / len(merged.columns)

merged.iplot(kind='bar', mode='group', subplots=True, shared_yaxis=True, legend='top', colors=cf_colors,
             title=f'Posizioni aperte in un dato momento [by Open] [non_one={non_one:.1%}]')

### Streak Analysis

Quando si studiano le streak, è importante tenere a mente che un trade (2) può essere stato aperto prima che il trade precendente (1) venisse chiuso, e quindi in realtà non si saprebbe l'esito del trade (1) nel momento in cui si apre (2). Come si vede dal grafico sopra, nel caso di Daniel questo avviene raramente (6.1% dei casi).

### Consideriamo questa strategia di rischio (1):
- Se abbiamo perso 1 volta: rischio=0.75%
- Se abbiamo perso 3 volte di fila: rischio=0.75%
- Se abbiamo vinto 1 volta: rischio=2%
- Se abbiamo vinto 3 volta di fila: rischio=2%

In [31]:
risk = {-1: 0.75, -3: 0.75, 1: 2, 3: 2}

musketeers = {'normal': result_by_close_given_risk(map_risk_to_streak(df1, risk)),
              'max_2': result_by_close_given_risk(map_risk_to_streak(reduce_daily_trades(df64_by_o, 2), risk)),
              'max_3': result_by_close_given_risk(map_risk_to_streak(reduce_daily_trades(df64_by_o, 3), risk))}

merged = merge(musketeers)

merged.cumsum().iplot(kind='scatter', connectgaps=True, interpolation='hv', colors=cf_colors, fill=False,
                      title='Return (%) [by Close]', xTitle='index', yTitle='%', legend='top', width=1.5, zerolinecolor='grey')

In [32]:
df = merged.copy().groupby(pd.Grouper(freq='M')).sum().reset_index()
df['close'] = df.close.dt.strftime('%b %Y')
df.iplot(kind='bar', x='close', opacity=1, title='Risultato mensile (%) [by Close]',
         yTitle='%', zerolinecolor='grey', colors=cf_colors, xTitle='month', legend='top')

In [33]:
merged = merge({'normal': drawdown(musketeers['normal']),
                'max_2': drawdown(musketeers['max_2']),
                'max_3': drawdown(musketeers['max_3'])})

merged.reset_index(drop=True).iplot(kind='scatter', connectgaps=True, interpolation='hv', colors=cf_colors, fill=False,
                      title='Drawdown (%) [by Close]', xTitle='index', yTitle='%', legend='bottom', width=1.5)

## Esiti

In [34]:
df64_2 = reduce_daily_trades(df64_by_o, 2)
df64_2['result'] = df64_2.result * df64_2.side.map({'BUY': 1, 'SELL': 0})

df64_2_i = reduce_daily_trades(df64_by_o, 2)
df64_2_i['result'] = (df64_2_i.result * df64_2_i.side.map({'BUY': 1, 'SELL': -1})).apply(lambda x: -1 if x < -0.9 else x)

df64_3 = reduce_daily_trades(df64_by_o, 3)
df64_3['result'] = (df64_3.result * df64_3.side.map({'BUY': 1, 'SELL': -1})).apply(lambda x: -1 if x < -0.9 else x)

variations = {
    '60/40_2_no_sell': df64_2,
    '60/40_2_inv_sell': df64_2_i,
    '60/40_3_inv_sell': df64_3,
    '60/40_6' : reduce_daily_trades(df64_by_o, 6)
}

merged = pd.concat({k: challenge_pass(v).value_counts() for k, v in variations.items()}, axis=1).dropna(how='all')

In [35]:
df = pd.DataFrame(index='30d avg ret, 30d std, 30d avg ret + std, 30d avg ret - std'.split(',')
                       +'7d avg ret, 7d std, 7d avg ret + std, 7d avg ret - std'.split(','))
df['unit'] = 'R'
weekly = []
monthly = pd.DataFrame()
for name, x in variations.items():
    by_D = x.set_index('close').sort_index().groupby(pd.Grouper(freq='D')).sum()
    mtd = by_D.result.rolling(30).sum()
    wtd = pd.DataFrame(by_D.result.rolling(7).sum().reset_index(drop=True))
    wtd['version'] = name
    weekly.append(wtd)
    monthly[name] = mtd

In [36]:
merged.apply(lambda x: x/x.sum()).reset_index().iplot(
    kind='bar', x='pass', colors=cf_colors, legend='top', opacity=1, yTitle='probability',
    title='Probabilità di passare Challenge FTMO', zerolinecolor='grey')

In [37]:
merged = pd.concat({
        k: challenge_pass(v, max_dd=-0.12, target=0.08).value_counts() for k, v in variations.items()},
    axis=1).dropna(how='all')

In [38]:
merged.apply(lambda x: x/x.sum()).reset_index().iplot(
    kind='bar', colors=cf_colors, x='pass', legend='top', opacity=1, yTitle='probability',
    title='Probabilità di passare Challenge MFF', zerolinecolor='grey')